In [2]:
import  h5py
import numpy as np
import random
from scipy.misc import imread,imsave,imresize 


Tools function to process the data

In [3]:
def read_normalize_img(dataset,i):
    i=str(i)
    path='dataset/'+dataset+'/'+i+'.jpg'
    img=imread(path)
    img=imresize(img,(300,300))
    #cast int to float32!!!
    img=img.astype(np.float32)
     #normalization(-1,1)
    img=(img-128)/128
    return img


def save(dataset,file_i):
    file_i=str(file_i)
    file_path="hdf5/"+dataset+"/"+file_i+".hdf5"
    try:
        if dataset=='train': 
            f=h5py.File(file_path,"w")
            f[file_i]=np.array(temp_train)
        elif dataset=='validate':
            f=h5py.File(file_path,"w")
            f[file_i]=np.array(temp_validate)
        else:
            f=h5py.File(file_path,"w")
            f[file_i]=np.array(temp_test)
    except Exception:
        f.close()
        
                

def load(dataset,file_i):
    file_i=str(file_i)
    file_path="hdf5/"+dataset+"/"+file_i+".hdf5"
    try:
        f=h5py.File(file_path,"r")
        #Datesets class，when slice return nmarray
        return f[file_i]    
    except Exception:
        f.close()


        
#prepare the train data
def load_batch_train(file_i,batch_size):
        #load file_i
        #hdf5 Datesets class
        train_features=load('train',file_i)
        return batch_train(train_features,train_labels,batch_size)
    
#a generator,yield the batches of feature and label for one file   
#. 
def batch_train(features, labels, batch_size):
    
    """
    Split features and labels into batches
    """
        #start：[0,batch_size,2 batch_size,.....]
        #batch data:0-batch_size-1   batch_size_2 batch-size-1......  
    for start in range(0, len(features), batch_size):        
        #array slicing,not include the last element
        #strat+batch_size should not larger than len of feature,once exceed，the end is the len
        end = min(start + batch_size, len(features))
        #when slice return nparray
        yield features[start:end], labels[start:end] 
        
  

#randomly choose 10% train set as validate set
random.seed(1)
validate_sample_list=random.sample(range(1,2296),229)
print('validate set:',len(validate_sample_list))


validate set: 229


Save train and validate features data on disk

In [26]:
temp_train=[]
temp_validate=[]
#the num of train pics
train_count=0
#the num of batch
count=1
#save the train set in disk
for i in range(1,2296):#2296
    if i not in validate_sample_list:
        train_count+=1
        #save disk the batch of pics
        if train_count%256==0:            
            #write in disk,empty list and read next pic
           # print('begin to save',count,'train')
            save('train',count)
            print('have saved',count,'train')
            temp_train=[]
            temp_train.append(read_normalize_img('train',i).tolist()) 
           # print(np.array(temp_train).shape)
            count+=1
                       
        #continue appending
        else:
            temp_train.append(read_normalize_img('train',i).tolist())
            
    else:
        temp_validate.append(read_normalize_img('train',i).tolist())

#save the last batch pics to disk
save('train',count)
print('have saved last train')
del temp_train

#save the validate set to disk
save('validate',1) 
del temp_validate

print('finished!!!')



have saved 1 train
have saved 2 train
have saved 3 train
have saved 4 train
have saved 5 train
have saved 6 train
have saved 7 train
have saved 8 train
have saved last train
finished!!!


Save test features data on disk

In [27]:
#save test data
temp_test=[]
count=1
for i in range(1,1532):
    if i%256==0:
        #write in disk,empty list and read next pic
        save('test',count)
        temp_test=[]
        temp_test.append(read_normalize_img('test',i).tolist())
        count+=1
    else:                     
        temp_test.append(read_normalize_img('test',i).tolist())
#save the last batch pics to disk
save('test',count)    
temp_test=None
print('finished!!!')

finished!!!


Prepare the labels data

In [3]:
#read label
import pandas as pd
label=pd.read_csv('train_labels.csv')
#type(label["name"])

In [4]:

#validate label array
#validate_label=label.values[(np.array(validate_sample_list)-1).tolist(),:][:,1:2].reshape(-1)

validate_label_list=label[label.name.isin(validate_sample_list)].invasive.values

train_label_list=label[~label.name.isin(validate_sample_list)].invasive.values
"""
def one_hot(label_list):
    temp_list=[]
    for i in label_list:
        if i==1:
            temp_list.append([1,0])
        else:
            temp_list.append([0,1])
    return temp_list
            
one_hot(validate_label_list)
"""    

 

#list to array [batch,1]
validate_labels=np.array(validate_label_list).reshape([-1,1])
                                                  
train_labels=np.array(train_label_list).reshape([-1,1])

print('validate_label: ',validate_labels.shape)
print('train_label:' ,train_labels.shape)
#onehot encoding[batch,2]
#def one_hot(y):
    


validate_label:  (229, 1)
train_label: (2066, 1)


Build the network


1.Prepare the placeholders


In [36]:
import tensorflow as tf
#clear the graph！！！before add to it
# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()
# create Inputs : placeholder,placeholder define the shape uncertaity thing
#batches of image input
x = tf.placeholder(tf.float32,shape=(None,300,300,3),name='x')
#batches label
y = tf.placeholder(tf.float32,shape=(None,1),name='y')

keep_prob =tf.placeholder(tf.float32,name='keep_prob')


2 Encapsulate the layers: genralize to all kinds of input and output


Convolution and Max Pooling Layer

In [37]:
#input a tensor,execute the convolution and pooling,return a new tensor
def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides,conv_padding='SAME'):
    """
    Apply convolution then max pooling to x_tensor
    :param x_tensor: TensorFlow Tensor input(4 dimension) 
    :param conv_num_outputs: Number of output feature maps of this convolutional layer
    :param conv_ksize: kernal size 2-D Tuple for the convolutional layer
    :param conv_strides: Stride 2-D Tuple for convolution
    :param pool_ksize: kernal size 2-D Tuple for pool
    :param pool_strides: Stride 2-D Tuple for pool
    : return: A tensor that represents convolution and max pooling of x_tensor (4 dimension) 
    """
    # first Max-Pooling -> finally ReLU ->output
    #tf.truncated_normal；tf.zeros
    #bulit parameter Variable and initialize the weight and bias of filter
    #weight:[ksize,ksize,depth,output],all weight for all filters（or feature maps）
    #one weight represent a filter，one out put one filter，conv_num_outputs group of weights
    cw=tf.Variable(tf.truncated_normal([conv_ksize[0],conv_ksize[1],x_tensor.get_shape().as_list()[3],conv_num_outputs],stddev=0.04))
    #all bias of the conv layer
    cb=tf.Variable(tf.zeros([conv_num_outputs]))
    #return 4 dimension output tensor[batch,conv_size1,conv_size2,conv_num_outputs]
    conv=tf.nn.conv2d(x_tensor,cw,strides=[1,conv_strides[0],conv_strides[1],1],padding=conv_padding)
    conv=tf.nn.bias_add(conv,cb)
    
    #return 4 dimension
    output=tf.nn.relu(conv)
    output=tf.nn.max_pool(output,ksize=[1,pool_ksize[0],pool_ksize[1],1],strides=[1,pool_strides[0],pool_strides[1],1],padding='SAME')
    
    return output 
    

Flatten Layer：change a tensor from 4-D to 2-D 

In [38]:
#flatteb a tensor for a fully connected layer
#finally we want[Batch Size, output],one batch one output
#so,flatten to (Batch Size, Flattened Image Size) multiple weight:[Flattened Image Size,output] get the result!
def flatten(x_tensor):
    """
    Flatten a x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...),  ... are the image dimensions.
    batchsize is uncertain!!
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    shape=x_tensor.get_shape().as_list()
    return tf.reshape(x_tensor,[-1,shape[1]*shape[2]*shape[3]])

Fully-Connected Layer

In [39]:
#input a 2D tensor,execute the forward pass of general neuron network，output a 2D tensor[batch,output]
def fully_conn(x_tensor, num_outputs):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    
    fw=tf.Variable(tf.truncated_normal([x_tensor.get_shape().as_list()[1],num_outputs],stddev=0.04))
    fb=tf.Variable(tf.zeros([num_outputs]))
    fo=tf.add(tf.matmul(x_tensor,fw),fb)
    output=tf.nn.relu(fo)
    return output

Output Layer

In [40]:
#input 2D tensor [batch,output]to 2D tensor[batch,logit]
def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    ow=tf.Variable(tf.truncated_normal([x_tensor.get_shape().as_list()[1],num_outputs],stddev=0.04))
    ob=tf.Variable(tf.zeros([num_outputs]))
    logit=tf.add(tf.matmul(x_tensor,ow),ob)
    return logit

Built Convolutional Model

In [41]:
from math import ceil
# Remove previous weights, bias, inputs, etc..



#out = ceil(float(75/ float(2)))

#from input x to output logit function
def conv_net(x, keep_prob):
    
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
     

    #built convolution layer
    #output:[148,148]
    conv_ksize1=(5,5)
    conv_strides1=(2,2)  
    
    conv_ksize2=(3,3)
    conv_strides2=(1,1) 
  
    conv_ksize3=(3,3)
    conv_strides3=(1,1)
   
    conv_ksize4=(3,3)
    conv_strides=(1,)
    
    conv_ksize5=(3,3)
    conv_strides=(1,1)
    
    pool_ksize=(2,2)
    pool_strides=(2,2)
    
   # cl=conv2d_maxpool(x, 32, conv_ksize1, conv_strides1, pool_ksize, pool_strides,'SAME')
    
    cl=conv2d_maxpool(x, 32, conv_ksize1, conv_strides1, pool_ksize, pool_strides,'SAME')
    #cl=tf.nn.dropout(cl,keep_prob)
    cl=conv2d_maxpool(cl, 64, conv_ksize3, conv_strides3, pool_ksize, pool_strides,'SAME')
   
    cl=conv2d_maxpool(cl, 128, conv_ksize4, conv_strides3, pool_ksize, pool_strides,'SAME')
  
    #cl=conv2d_maxpool(cl, 512, conv_ksize5, conv_strides3, pool_ksize, pool_strides,'VALID')
    
    #connect fully-connceted layer
    cl_2d=flatten(cl)
    
    fcl=fully_conn(cl_2d, 521)
    fcl=tf.nn.dropout(fcl,keep_prob)
    fcl=fully_conn(fcl, 128)
    fcl=tf.nn.dropout(fcl,keep_prob)
    #fcl=fully_conn(fcl, 128)
 
    
    #  Output Layer
    logit=output(fcl, 1)
    return logit

# The Model！！！1 [batch,1]]
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
#logistics loss for one output

y=tf.cast(y,tf.float32)
logits=tf.cast(logits,tf.float32)
part_1=tf.multiply(y,tf.log(tf.sigmoid(logits)))
part_2=tf.multiply(1-y,tf.log(1-tf.sigmoid(logits)))
#tf.subtract(tf.constant(1,dtype=tf.float32),y)
#logistics loss function
cost = tf.multiply(tf.constant(-1,dtype=tf.float32),tf.reduce_mean(tf.add(part_1,part_2)))

#updata the weight
#optimizer = tf.train.AdadeltaOptimizer().minimize(cost)
#optimizer=tf.train.GradientDescentOptimizer().minimize(cost)#learning_rate=0.00001
optimizer = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(cost)#0.0016
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
#optimizer = tf.train.AdadeltaOptimizer().minimize(cost)#learning_rate=0.0005
"""
解决方案1:

cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y_conv,1e-10,1.0)))  
解决方案2:(推荐)

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv + 1e-10)
"""
#compute any features' Accuracy
#logist,y are 2 dimension[batch,value].same shape.1 represent row dimension
#argmax return tensor array，if the position of max logist and max label are same，predict correctly return true 
#correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
correct_pred=tf.equal(tf.greater(logits,0.5),tf.equal(y,1.0))
#correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
#cast boolean array to float array
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

Show the training stats

In [42]:
#after optimizer and update the weight，run the accuracy on the same train batch data！
#depend on logit and y
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    """
    Use the present model to compute loss and validation accuracy！！！
    Only by using x,y
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    
    #when evaluate we use the certain model
    cost=session.run(cost,feed_dict={x:feature_batch,y:label_batch,keep_prob:1.0})
        
    #use validate set!!!!
    validate_features=load('validate',1)
    #print('logits:',session.run(logits,feed_dict={x:validate_features,keep_prob:1.0}))        
    #print('logits>0.5:',session.run(tf.greater(logits,0.5),feed_dict={x:validate_features,keep_prob:1.0}))
    #print('y equal to 1:',session.run(tf.equal(y,1),feed_dict={y:validate_labels,keep_prob:1.0}))
    #print('correct_table',session.run(correct_pred,feed_dict={x:validate_features,y:validate_labels,keep_prob:1.0}))
    validate_accuracy=session.run(accuracy,feed_dict={x:validate_features,y:validate_labels,keep_prob:1.0})  
    train_accuracy=session.run(accuracy,feed_dict={x:feature_batch,y:label_batch,keep_prob:1.0}) 
    
    print('loss:{},validate_accuracy:{},train_accuracy:{}'.format(cost,validate_accuracy,train_accuracy))
    

Tune Hyperparameters

In [ ]:
epochs = 20
batch_size = 32
keep_probability = 0.5

Train all

In [ ]:

with tf.Session() as sess:
    
    
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    
    for epoch in range(1,epochs+1):
        print('Epoch'+str(epoch)+':',end='')
         # Loop over all files
        n_files = 7
        for i in range(1, n_files + 1):
            for feature_batch,label_batch in load_batch_train(i,batch_size):                
                sess.run(optimizer,feed_dict={x:feature_batch,y:label_batch,keep_prob:keep_probability})
                
        #after train,test the present model 
        print_stats(sess, feature_batch, label_batch, cost, accuracy)

    
      # Save Model
      #loss func value 跟 validate_accuracy,没有直接的关系，不需要太关注它
    saver = tf.train.Saver()
    save_path = saver.save(sess, 'saved_model/')


Epoch1:loss:0.5843467712402344,validate_accuracy:0.4585152864456177,train_accuracy:0.1875
Epoch2:loss:0.5575419664382935,validate_accuracy:0.6681222915649414,train_accuracy:0.65625
Epoch3:loss:0.5523676872253418,validate_accuracy:0.6593886613845825,train_accuracy:0.6875
Epoch4:loss:0.5410900712013245,validate_accuracy:0.6375545859336853,train_accuracy:0.75
Epoch5:loss:0.5293526649475098,validate_accuracy:0.6462882161140442,train_accuracy:0.78125
Epoch6:loss:0.5394490361213684,validate_accuracy:0.7030567526817322,train_accuracy:0.78125
Epoch7:loss:0.542778730392456,validate_accuracy:0.6768559217453003,train_accuracy:0.71875
Epoch8:loss:0.5045607089996338,validate_accuracy:0.7030567526817322,train_accuracy:0.75
Epoch9:

Built Convolutional Model

In [ ]:
#1 checking the model on one batch
#2 spend litte time to iterate  the model for a better accuracy
"""
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(1,epochs+1):
        for feature_batch,label_batch in load_batch_train(1,batch_size):
            train(sess, optimizer, keep_probability, feature_batch, label_batch)
        print('Epoch'+str(epoch)+':')
        #after train,test the present model 
        print_stats(sess, feature_batch, label_batch, cost, accuracy)
"""